In [2]:
from neo4j import GraphDatabase

### Connection to Neo4j KG database

In [3]:
# Neo4j LLM Builder
# NEO4J_URI = "neo4j+s://fcc22d75.databases.neo4j.io"
# NEO4J_USERNAME = "neo4j"
# NEO4J_PASSWORD = "Abdo3VjaAxAfAZkkSWvRsQZN_IJ4EWTqUP9RTg0VwIc"

# local connection
# NEO4J_URI = "bolt://localhost:7687"
# NEO4J_USERNAME = "neo4j"
# NEO4J_PASSWORD = "password123"

# Adrien's KG 
NEO4J_URI="neo4j+s://4d4a6555.databases.neo4j.io:7687"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="cTzNG9WsDNDSZW8M7U44wcmsbvsuTnPzjChLNi9qt9E"

# Connect to Neo4j
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

print("Connected to Neo4j")

Connected to Neo4j


### Data exploration

Basic data exploration of the synthetic knowledge graph

In [4]:
def run_query(query):
    with driver.session() as session:
        result = session.run(query)
        return [record for record in result]

In [5]:
# 1. Count number of nodes and relationships
node_count = run_query("MATCH (n) RETURN count(n) AS total_nodes")
relationship_count = run_query("MATCH ()-[r]->() RETURN count(r) AS total_relationships")
print("Total Nodes:", node_count[0]['total_nodes'])
print("Total Relationships:", relationship_count[0]['total_relationships'])

Total Nodes: 432
Total Relationships: 389


In [6]:
# 2. Get distinct node labels
node_labels = run_query("CALL db.labels() YIELD label RETURN label")
print("Node Labels:", [record['label'] for record in node_labels])
# 3. Get distinct relationship types
relationship_types = run_query("CALL db.relationshipTypes() YIELD relationshipType RETURN relationshipType")
print("Relationship Types:", [record['relationshipType'] for record in relationship_types])

Node Labels: ['Chunk', '__Entity__', 'Skill']
Relationship Types: ['IS_PRE_REQUISITE_OF']


In [7]:
# 4. Get top 5 nodes with most relationships
top_nodes = run_query("MATCH (n)-[r]-() RETURN n, count(r) AS degree ORDER BY degree DESC LIMIT 5")
print("Top 5 Most Connected Nodes:")
for record in top_nodes:
    print(record['n'])

Top 5 Most Connected Nodes:
<Node element_id='4:728ea546-571a-4682-a668-69c89dc4f483:401' labels=frozenset({'Skill'}) properties={'materials': ['2687f888-b94e-4324-9ad7-c1c8a2de6ba4', 'adf131fe-17dd-41cd-ad1f-7cb8a7274427', '6eb0a701-50be-4c46-850e-7265b7179089', '9de08207-70d7-46a2-83c7-3859f8d5d19c', 'ea1dd49d-730b-450a-b649-6d67e9a3fe11', '19876630-6bc5-4202-b6e3-f76259a43e2b', '33029dda-1212-4277-8975-6b1c77896dd5', 'c1381bbe-5fb7-4fe8-8e54-b9a20fe503e7', '80aba047-df03-4600-8a1a-984540ccd1f0', '961e339c-d4d1-4c8e-bab3-32bbd989a315', '02d4fb5d-74e0-49f8-880f-c8e13d62fdb0', 'fae385f2-0908-4265-9a41-4ce8b4f04699', '07f76538-308b-4459-918c-d8760285850e', '7c7b9c7d-b7e6-4ab9-ab39-971ab86eebea', 'bc544988-8f43-4045-8cc6-bb64bc61f3e8'], 'name': 'machine learning'}>
<Node element_id='4:728ea546-571a-4682-a668-69c89dc4f483:457' labels=frozenset({'Skill'}) properties={'materials': ['adf131fe-17dd-41cd-ad1f-7cb8a7274427', '0d72b55e-f495-438d-af49-a91d91aa3a9e', '02a79db9-792e-43b1-99d5-1c4ec

In [8]:
def get_textual_content(node_id):
    with driver.session() as session:
        query = """
        MATCH (n)
        WHERE n.id = $node_id
        RETURN n.text  // Replace 'text' with the correct textual property name
        """
        result = session.run(query, node_id=node_id)
        node = result.single()  # Returns the first match
        
        if node:
            return node['n.text']  # The textual content of the node
        else:
            return None

# Get the textual content of the node 
textual_content = get_textual_content('AI')
print(textual_content)

None


In [ ]:
def get_longest_textual_node():
    with driver.session() as session:
        query = """
        MATCH (n)
        WHERE n.text IS NOT NULL  // Check if the 'text' property is not null
        WITH n, size(n.text) AS text_length  // Calculate the length of 'text'
        ORDER BY text_length DESC  // Order nodes by text length in descending order
        LIMIT 1  // Get the node with the longest text
        RETURN n, text_length
        """
        result = session.run(query)
        node = result.single()  # Returns the first match (node with longest text)

        if node:
            return node['n'], node['text_length']  # Return node and its textual length
        else:
            return None

# Get the node with the longest textual content
longest_node, text_length = get_longest_textual_node()

# Handle the case where no node was found
if longest_node is not None:
    print(f"Node with the longest text: {longest_node}")
    print(f"Text length: {text_length}")
    print(f"Text: {longest_node['text']}")  
else:
    print("No nodes with textual content found.")

Node with the longest text: <Node element_id='4:f1d35087-bbee-462c-88fd-a0645044e884:19845' labels=frozenset({'Chunk'}) properties={'fileName': 'Algorithmic party platforms in the United States.txt', 'content_offset': 7156, 'length': 674, 'text': ' in political platforms also raises ethical and transparency concerns, particularly regarding the authenticity of dynamically adjusted messaging and the potential for voter manipulation. Addressing these challenges is crucial to maintaining voter trust and the integrity of the democratic process. In summary, AI significantly shapes political platforms in real time by providing campaigns with the tools to analyze voter sentiment, segment audiences, and adjust strategies dynamically. While offering substantial benefits in responsiveness and engagement, it is imperative to navigate the accompanying ethical considerations to ensure the responsible use of AI in political', 'embedding': [-0.026420066133141518, -0.04907726123929024, -0.0475012250244